In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [92]:
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
def scrollDown(driver, reviewTotal, wait):
    x = len(driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']"))
    lastx = x - 1
    i = 1
    repCount = 0
    print("scrolling...")
    # while we expect more reviews and we haven't repeated too much
    while ((x < reviewTotal) and (repCount < 3)):
        # pause to not look like a bot
        pauseScroll(wait)
        # select list of reviews
        elements = driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']")
        # find the last visible review and go to it
        current_last = elements[-1]
        current_last.location_once_scrolled_into_view
        # wait until page loads
        WebDriverWait(driver, 30).until(ec.invisibility_of_element_located((By.XPATH, "//div[@class='jfk-activityIndicator-icon']")))
        # take the previous review count and replace it with the new one
        lastx = x
        x = len(driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']"))
        # if more reviews are not added to the visible reviews, mark repetition
        if (lastx == x):
            repCount += 1
        else:
            repCount = 0
            
    print("finished scrolling, found", x, "reviews")

In [71]:
def getReviewTotal(driver):
    raw = driver.find_element_by_xpath("//span[@class='hqzQac']").text
    count = int(raw.split(" ")[0])
    print("expecting", count, "reviews")
    return count

In [72]:
import numbers
import time
def pauseScroll(wait):
    waitTime = 0
    # if argument is a list
    if (isinstance(wait, list)):
        # check if its length is valid
        if (len(wait) > 2) or (len(wait) < 1):
            raise ValueError
        # if length is valid get random number between vals
        low = wait[0]
        high = wait[1]
        # check for improper types, fix if possible
        if not isinstance(low, numbers.Number):
            if low.isnumeric():
                low = float(low)
            else:
                raise ValueError
        if not isinstance(high, numbers.Number):
            if high.isnumeric():
                high = float(high)
            else:
                raise ValueError
        waitTime = randTime(low, high)
    # if argument is a number
    elif (isinstance(wait, numbers.Number)):
        # get the number
        waitTime = wait
    # if argument is a string
    elif (isinstance(wait, str)):
        # if argument can be parsed as a number
        if (wait.isnumeric()):
            # get time as a number
            waitTime = float(wait)
        else:
            raise ValueError
    time.sleep(waitTime)

In [5]:
from random import random
def randTime(low, high):
    val = random()
    time = low + (val  * (high - low))
    return time

In [85]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
# note that this assumes scrolling has already occurred
def scrapeFromList(driver, key, name=True, stars=True, text=True, timeSince=True):
    elements = driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']")
    columnsBin = [name, stars, text, timeSince]
    columnsNames = ["name", "stars", "text", "timeSince"]
    columnsPresent = [x for i, x in enumerate(columnsNames) if columnsBin[i]] # gets included columns
    print("preparing to scrape location with key", key)
    
    # text body
    textList = []
    if (text):
        textPath = ".//div[@style='vertical-align:top']//div[@class='Jtu6Td']//span[@jscontroller='P7L8k']"
        textPathFullExtension = "//span[@class='review-full-text']"
        textList = [0]*len(elements)
        i = 0
        for e in elements:
            try:
                textPartial = e.find_element_by_xpath(textPath+textPathFullExtension)
                e.find_element_by_xpath(textPath+"//span[@role='button']").click()
                textList[i] = e.find_element_by_xpath(textPath).text
            except NoSuchElementException:
                textList[i] = e.find_element_by_xpath(textPath).text
            i += 1
        
    # name
    nameList = []
    if (name):
        namePath = ".//div[@class='TSUbDb']"
        nameList = [e.find_element_by_xpath(namePath).text for e in elements]
    
    # stars
    starsList = []
    if (stars):
        starsPath = ".//div[@style='vertical-align:top']//div[@class='PuaHbe']//g-review-stars[@style='padding-right:7px']//span[@class='Fam1ne EBe2gf']"
        i = 0
        starsList = [0]*len(elements)
        for e in elements:
            raw = e.find_element_by_xpath(starsPath).get_attribute("aria-label")
            starCount = raw.split(" ")[1] # the number of stars
            starsList[i] = float(starCount)
            i += 1
    
    # timeSince
    timeSinceList = []
    if (timeSince):
        timeSincePath = ".//div[@style='vertical-align:top']//div[@class='PuaHbe']"
        timeSinceList = [e.find_element_by_xpath(timeSincePath).text for e in elements]
    
    # put df together
    columnsBin = [name, stars, text, timeSince]
    columnsNames = ["name", "stars", "text", "timeSince"]
    columnsPresent = [x for i, x in enumerate(columnsNames) if columnsBin[i]] # gets included columns
    columns = [nameList, starsList, textList, timeSinceList]
    columns = [c for c in columns if len(c) > 0]
    data = {c:columns[i] for i,c in enumerate(columnsPresent)}
    df = pd.DataFrame(data)
    print("scraped location with key", key, "with", len(df), "reviews")
    
    return df

In [141]:
import datetime
import pandas
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
def scrapeFromURLs(urls, checkAddress=False, combine=True, wait=[2,3]):
    # check that urls are dicts
    if not (isinstance(urls, dict)):
        raise ValueError
        
    # start selenium stuff
    driver = webdriver.Chrome('./chromedriver')
    
    dfs = [0]*len(urls)
    
    # for each id
    r_count = 0
    i = 0
    status = [0]*len(urls)
    for id in urls:
        
        # go to url
        url = urls.get(id)
        driver.get(url)
        
        # check that address is correct
        # note this will assume the second url parameter is address number, last is state, second to last is town
        if checkAddress:
            parameters = url.split('=')[1].split('+') # gets the url parameters after the '=' in a list
            addr_number = parameters[1].split(' ')[0] # gets the address number
            town = parameters[-2].lower() # gets the town
            state = parameters[-1].lower() # gets the state
            addr = driver.find_element_by_xpath("//span[@class='LrzXr']").text.split(',') # gets the found address as list
            # if a check is not met, do not scrape the location, i.e. put a blank df in its final place and do not mark as done
            found_addr_number = addr[0].split(' ')[0]
            found_town = addr[1].lstrip(' ').lower()
            found_state = addr[2].lstrip(' ').split(' ')[0].lower()
            if found_addr_number != addr_number:
                dfs[i] = pd.DataFrame({})
                i += 1
                print("failed to scrape location with key",id,"due to unmatched address number, continuing...")
                continue
            elif found_town != town:
                dfs[i] = pd.DataFrame({})
                i += 1
                print("failed to scrape location with key",id,"due to unmatched town, continuing...")
                continue
            elif found_state != state:
                dfs[i] = pd.DataFrame({})
                i += 1
                print("failed to scrape location with key",id,"due to unmatched state, continuing...")
                continue
        
        driver.find_element_by_class_name("hqzQac").click()
        WebDriverWait(driver, 45).until(ec.presence_of_element_located((By.XPATH, "//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']")))
        
        # scroll to bottom
        scrollDown(driver, getReviewTotal(driver), wait)
        
        # scrape from loaded list
        # if a scrape fails, move onto the next one
        try:
            df = scrapeFromList(driver, id)
        except:
            dfs[i] = pd.DataFrame({})
            i += 1
            print("failed to scrape location with key",id,", continuing...")
            continue
        df["key"] = str(id)
        
        dfs[i] = df
        r_count += len(df)
        status[i] = 1
        i += 1
        
    # combine dfs if necessary
    if (combine):
        df = dfs[0]
        # append all other dfs to the first one, then save to csv
        for i in dfs[1:]:
            df = df.append(i, ignore_index=True)
        r_count_final = len(df)
        
        df.to_csv("combined_scrape_"+str(datetime.datetime.now()).replace(' ','_').replace(':','_').replace('.','_')+".csv",index=False)
    else:
        for i in dfs:
            i.to_csv("scraped_"+str(datetime.datetime.now()).replace(' ','_').replace(':','_').replace('.','_')+".csv",index=False)
    
    # for testing if rows are lost
    print("sum of all rows:",r_count)
    print("total rows:",r_count_final)
    
    driver.quit()
    
    return status

In [8]:
# TODO
# make scrapeFromURLs take in a filename convention and have csvs save as it
# add check for correct address
# check that reviews button is there, i.e. check whether there is actually a location found

In [75]:
import pandas as pd
def extractURLs_csv(csv, searchTerms, key, firstIndex, lastIndex):
    baseString = "https://www.google.com/search?q="
    
    # read in csv (maybe add value checks later)
    df = pd.read_csv(csv)
    
    # makes sure key is a list so it can be added to searchTerms
    if (not isinstance(key, list)):
        key = [key]
    
    search = df.loc[list(range(firstIndex,lastIndex+1)), searchTerms + key] # gets df with only search terms in desired rows
    
    search["url"] = baseString + search[searchTerms[0]] # column of base plus first search term
    # creates a column of url by adding on all other terms
    for s in searchTerms[1:]:
        search["url"] = search["url"] + "+" + search[s].astype(s)
        
    # converts key back into string
    key = key[0]
    
    # returns dictionary with key of key and value of url
    return dict(zip(list(search[key]),list(search["url"])))
    

In [76]:
import pandas as pd
import xlrd
def extractURLs_excel(xlsx, searchTerms, key, firstIndex, lastIndex):
    baseString = "https://www.google.com/search?q="
    
    # read in csv (maybe add value checks later)
    df = pd.read_excel(xlsx)
    
    # makes sure key is a list so it can be added to searchTerms
    if (not isinstance(key, list)):
        key = [key]
    
    search = df.loc[list(range(firstIndex,lastIndex+1)), searchTerms + key] # gets df with only search terms in desired rows
    
    search["url"] = baseString + search[searchTerms[0]] # column of base plus first search term
    # creates a column of url by adding on all other terms
    for s in searchTerms[1:]:
        search["url"] = search["url"] + "+" + search[s].astype(str)
        
    # converts key back into string
    key = key[0]
    
    # returns dictionary with key of key and value of url
    return dict(zip(list(search[key]),list(search["url"])))

In [11]:
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_small.xlsx"
indices = [17, 19]
filePath = "C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes"

In [147]:
import openpyxl
def simpleScraper(sourceFile, searchTerms, key, firstIndex, lastIndex, wait=[2,3], combine=True, statusCol=None, overWrite=False):
    
    # get url dict
    ext = sourceFile.split(".")[-1] # file extension
    urls = {}
    if (ext == "xlsx"):
        urls = extractURLs_excel(sourceFile, searchTerms, key, firstIndex, lastIndex)
    elif (ext == "csv"):
        urls = extractURLs_csv(sourceFile, searchTerms, key, firstIndex, lastIndex)
    else:
        raise ValueError
    
    # scrape and save
    status = scrapeFromURLs(urls, combine, wait)
    
    # mark source file
    if statusCol is not None:
        if (ext == "xlsx"):
            source = pd.read_excel(sourceFile)
        elif (ext == "csv"):
            source = pd.read_csv(sourceFile)
        else:
            raise ValueError
        # make status list and put into source file df
        status_final = ["Scraped" if s is 1 else "Failed" for s in status]
        source.loc[firstIndex:lastIndex,[statusCol]] = status_final
        if overWrite:
            if (ext == "xlsx"):
                source.to_excel(sourceFile, index=False)
            elif (ext == "csv"):
                source.to_csv(sourceFile, index=False)
            else:
                raise ValueError
        else:
            if (ext == "xlsx"):
                source.to_excel(sourceFile.split('.')[0]+'_.'+ext, index=False)
            elif (ext == "csv"):
                source.to_csv(sourceFile.split('.')[0]+'_.'+ext, index=False)
            else:
                raise ValueError
            

In [146]:
simpleScraper(sourceFile, searchTerms, key, indices=indices, wait=[1,2], combine=False, filePath=filePath)

expecting 394 reviews
scrolling...
finished scrolling, found 393 reviews
preparing to scrape location with key 1000804781
scraped location with key 1000804781 with 393 reviews
sum of all rows: 393
total rows: 393


In [148]:
driver.quit()

In [6]:
import pandas as pd
data = {'Name':['Tom', 'nick', 'krish', 'jack'], 'Age':[20, 21, 19, 18]} 
df = pd.DataFrame(data) 

In [6]:
url = "https://www.google.com/search?q=Pizza Hut+10021004 Chestnut St+EMMAUS+PA"
driver = webdriver.Chrome('./chromedriver')

In [7]:
driver.get(url)
addr = driver.find_elements_by_xpath("//span[@class='LrzXr']")

In [20]:
indices = [0,2]
age_final = [23, 25]

In [15]:
filePath[-2:]

'es'

In [19]:
filePath + "\\"

'C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes\\'

In [20]:
import openpyxl
import pandas as pd

In [21]:
workfile = pd.read_excel("workfile.xlsx")

In [22]:
workfile.head()

,Company ID,Status,Industry ID,Industry Name,Company Name,Address 1,City,State,Zip,Listing Type,Country,Phone,Parent_ID,Parent Name,Parent City,google.com/search?q=Pizza%20Hut+Address 1+City+State
0,1.000805e+09,Scraped,51.0,Locations: Chain Restaurants,Pizza Hut,142 Highway 62,ASH FLAT,AR,72513,Location,United States of America,8.709947e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+142 Highway 62...
1,1.000805e+09,Scraped,51.0,Locations: Chain Restaurants,Pizza Hut,30 W 150 N,LAGRANGE,IN,46761-9027,Location,United States of America,2.604633e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+30 W 150 N+LAG...
2,1.000805e+09,Scraped,51.0,Locations: Chain Restaurants,Pizza Hut,1 David Dr,ESSEX JUNCTION,VT,05452-2826,Location,United States of America,8.028780e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1 David Dr+ESS...
3,1.000805e+09,Scraped,51.0,Locations: Chain Restaurants,Pizza Hut,10 Saint Francis Way,CRANBERRY TOWNSHIP,PA,16066-5130,Location,United States of America,7.247763e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+10 Saint Franc...
4,1.000805e+09,Scraped,51.0,Locations: Chain Restaurants,Pizza Hut,10 W Chestnut Hill Ln,REISTERSTOWN,MD,21136-3220,Location,United States of America,4.108337e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+10 W Chestnut ...


In [103]:
work_trimmed = workfile[workfile["Company ID"].notnull()]

In [46]:
unscraped = workfile[(workfile["Status"].isnull()) & (workfile["Company ID"].notnull())]

In [85]:
unscraped.iloc[0:500,:]

,Company ID,Status,Industry ID,Industry Name,Company Name,Address 1,City,State,Zip,Listing Type,Country,Phone,Parent_ID,Parent Name,Parent City,google.com/search?q=Pizza%20Hut+Address 1+City+State
787,1.000808e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1402 E Saint Patrick St,RAPID CITY,SD,57701-3937,Location,United States of America,6.053424e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1402 E Saint P...
788,1.000808e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,134 Yellow Creek Rd,EVANSTON,WY,82930-5224,Location,United States of America,3.077891e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+134 Yellow Cre...
789,1.000808e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1347 Kennedy Blvd,BAYONNE,NJ,07002-2245,Location,United States of America,2.014368e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1347 Kennedy B...
790,1.000808e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1350 Big A Rd,TOCCOA,GA,30577-6013,Location,United States of America,7.068861e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1350 Big A Rd+...
791,1.000808e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1350 Capitol Dr,PEWAUKEE,WI,53072-2589,Location,United States of America,2.626956e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1350 Capitol D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,1.000809e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,17539 Blanco Rd,SAN ANTONIO,TX,78232-1025,Location,United States of America,2.104934e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+17539 Blanco R...
1283,1.000809e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1754 N College Ave,FAYETTEVILLE,AR,72703-2605,Location,United States of America,4.795213e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1754 N College...
1284,1.000810e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1830 Kempsville Rd,VIRGINIA BEACH,VA,23464-6860,Location,United States of America,7.574800e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1830 Kempsvill...
1285,1.000810e+09,NaN,51.0,Locations: Chain Restaurants,Pizza Hut,1835 E Vienna St,ANNA,IL,62906-2033,Location,United States of America,6.188335e+09,1.000019e+09,YUM! Brands Inc.,LOUISVILLE,google.com/search?q=Pizza%20Hut+1835 E Vienna ...


In [86]:
import numpy as np

In [105]:
dfs = np.array_split(work_trimmed, len(work_trimmed)/500)
for df in dfs:
    first_index = str(df.first_valid_index())
    last_index = str(df.last_valid_index())
    filename = "workfile_" + first_index + "_" + last_index + ".xlsx"
    df.to_excel(filename,index=False)

In [102]:
str(dfs[0].last_valid_index())

'1291'

In [98]:
len(dfs)

10

In [107]:
# 9/29/20 3:21 scrape
# 256-306 (50 locations)
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_530_1059.xlsx"
indices = list(range(256,307))
filePath = "C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms, key=key, indices=indices, wait=[1,2], combine=False, statusCol="Status", filePath=filePath)

In [106]:
list(range(256,530))[-1]

529